In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
from PIL import Image
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

# ========== Set device ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== Load InceptionV3 model ==========
model = inception_v3(pretrained=True)
model.fc = torch.nn.Identity()  # Remove final classification layer
model.eval()
model.to(device)

# ========== Image transform ==========
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),           # Light augmentation
    transforms.RandomRotation(10),               # Light rotation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # InceptionV3 specific
])

# ========== Extract Features ==========
def extract_inception_features(img_tensor):
    with torch.no_grad():
        features = model(img_tensor)
    return features.cpu().numpy().flatten()

# ========== Loop over images ==========
soil_folder = '.../data/data/clahe_soil_images'  
features_list = []

for img_file in os.listdir(soil_folder):
    if not img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp','.gif')):
        continue

    img_path = os.path.join(soil_folder, img_file)
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)

    feat = extract_inception_features(img_tensor)
    features_list.append(feat)

# ========== Convert to NumPy array ==========
X = np.array(features_list)

# ========== Normalize and train One-Class SVM ==========
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm = OneClassSVM(kernel='rbf', nu=0.1, gamma='auto')
svm.fit(X_scaled)

# ========== Predict anomalies ==========
preds = svm.predict(X_scaled)  # +1 = normal, -1 = anomaly

# ========== Optional: Print results ==========
for img_file, pred in zip(os.listdir(soil_folder), preds):
    print(f"{img_file}: {'Normal' if pred == 1 else 'Anomalous'}")


In [ ]:
import joblib
joblib.dump(svm, '../trained_model/svm_model2.joblib')
joblib.dump(scaler, '../trained_model/scaler2.joblib')